# Amazon Recommender System - Data Exploration

## 1. Data loading

In [1]:
import numpy as np

data_path = '../data/raw/ratings_Beauty.csv'
column_names = ['UserId', 'ProductId', 'Rating', 'Timestamp']
data_raw = np.genfromtxt(data_path, delimiter = ",", dtype = str, encoding = 'utf-8', skip_header = 1)

## 2. Dataset overview

In [2]:
print(f"The dataset contains {len(data_raw)} rows")
print("Shape dataset:", data_raw.shape)
print("Column names:", column_names)

print("\nFirst five rows:")
for row in data_raw[:5]:
    print(row)

The dataset contains 2023070 rows
Shape dataset: (2023070, 4)
Column names: ['UserId', 'ProductId', 'Rating', 'Timestamp']

First five rows:
['A39HTATAQ9V7YF' '0205616461' '5.0' '1369699200']
['A3JM6GV9MNOF9X' '0558925278' '3.0' '1355443200']
['A1Z513UWSAAO0F' '0558925278' '5.0' '1404691200']
['A1WMRR494NWEWV' '0733001998' '4.0' '1382572800']
['A3IAAVS479H7M7' '0737104473' '1.0' '1274227200']


## 3. User & Product Overview

In [3]:
unique_users = np.unique(data_raw[:,0])
unique_products = np.unique(data_raw[:,1])

sparsity = 1 - len(data_raw)/(len(unique_users)*len(unique_products))
print("Total rating:", len(data_raw))
print("Unique users:", len(unique_users))
print("Unique products:", len(unique_products))
print("Sparsity:", sparsity)

Total rating: 2023070
Unique users: 1210271
Unique products: 249274
Sparsity: 0.999993294189133


#### Câu hỏi và nhận xét:
- Có bao nhiêu user và product duy nhất trong dataset?
    - Dataset có 2,023,070 rating, từ 1,210,271 users và 249,274 products.
- Tỷ lệ các cặp user-product chưa có rating là bao nhiêu? Ma trận user-product thưa thớt đến mức nào, ảnh hưởng gì tới việc chọn mô hình?
    - Sparsity ~ 0.99999: cực kỳ thưa thớt, nghĩa là hầu hết user-product pairs chưa được đánh giá.
    - Ma trận cực kỳ thưa thớt, nên khi chọn mô hình cần cân nhắc matrix factorization để tránh overfitting và xử lý long-tail problem.

## 4. Rating Distribution

In [4]:
ratings = data_raw[:,2].astype(float)
values, counts = np.unique(ratings, return_counts=True)

for v, c in zip(values, counts):
    print(f"Rating {v}: {c} ({c/len(ratings):.2%})")

avg_rating = np.mean(ratings)
print("\nAverage rating:", avg_rating)

Rating 1.0: 183784 (9.08%)
Rating 2.0: 113034 (5.59%)
Rating 3.0: 169791 (8.39%)
Rating 4.0: 307740 (15.21%)
Rating 5.0: 1248721 (61.72%)

Average rating: 4.149035871225415


#### Câu hỏi và nhận xét:
- Rating phân bố thế nào? Tỷ lệ các rating 1–5 ra sao?
    - Rating 5 sao chiếm áp đảo (~61.7%), 4 sao ~15%, các rating thấp (1-3 sao) chỉ chiếm ~23%.
- Rating trung bình là bao nhiêu? Dữ liệu có bias lệch về rating cao không?
    - Trung bình rating ~4.15, cho thấy dữ liệu nghiêng về high ratings.
    - Dataset có bias rating cao

## 5. User Activity

In [5]:
users = data_raw[:,0]

unique_users, user_freq = np.unique(users, return_counts=True)

print("Avg ratings per user:", np.mean(user_freq))
print("Max ratings per user:", np.max(user_freq))
print("Users with only 1 rating:", np.sum(user_freq == 1))
print("Percentage of users with only 1 rating: {:.2%}".format(np.sum(user_freq == 1)/len(unique_users)))

top_idx = np.argsort(user_freq)[-5:][::-1]  # chỉ số top 5
top_users = unique_users[top_idx]
top_counts = user_freq[top_idx]
print("\nTop 5 most active users:")
for u, c in zip(top_users, top_counts):
    print(u, c)

Avg ratings per user: 1.6715842980621696
Max ratings per user: 389
Users with only 1 rating: 887401
Percentage of users with only 1 rating: 73.32%

Top 5 most active users:
A3KEZLJ59C1JVH 389
A281NPSIMI1C2R 336
A3M174IC0VXOS2 326
A2V5R832QCSOMX 278
A3LJLRIZL38GG3 276


#### Câu hỏi và nhận xét:
- Mỗi user đánh giá trung bình bao nhiêu sản phẩm? Phần lớn user nằm ở nhóm ít đánh giá hay nhiều đánh giá?
    - Trung bình mỗi user đánh giá khoảng 1.67 sản phẩm, hầu hết người dùng chỉ đánh giá 1–2 sản phẩm.
- User nào hoạt động nhiều nhất và số lượng rating tối đa là bao nhiêu?
    - Số lượng đánh giá tối đa của một user là 389, nghĩa là một số người dùng rất tích cực.
- Bao nhiêu người dùng chỉ đánh giá 1 sản phẩm, và tỷ lệ so với tổng số user là bao nhiêu?
    - Có 887,401 người dùng (~73%) đánh giá chỉ 1 sản phẩm: dataset có nhiều người dùng ít hoạt động.
- Ai là 5 người dùng hoạt động nhiều nhất, và số lượng rating của họ so với phần còn lại như thế nào?
    - Top 5 user cho thấy họ cực kỳ tích cực, chênh lệch rõ so với phần lớn user.

## 6. Product Activity

In [6]:
products = data_raw[:,1]

unique_products, product_freq = np.unique(products, return_counts=True)

print("Avg ratings per product:", np.mean(product_freq))
print("Max ratings per product:", np.max(product_freq))
print("Products with only 1 rating:", np.sum(product_freq==1))
print("Percentage of products with only 1 rating: {:.2%}".format(np.sum(product_freq==1)/len(unique_products)))

top_idx = np.argsort(product_freq)[-5:][::-1]  # chỉ số top 5
top_products = unique_products[top_idx]
top_counts = product_freq[top_idx]
print("\nTop 5 most rated products:")
for p, c in zip(top_products, top_counts):
    print(p, c)

Avg ratings per product: 8.115848423822781
Max ratings per product: 7533
Products with only 1 rating: 103484
Percentage of products with only 1 rating: 41.51%

Top 5 most rated products:
B001MA0QY2 7533
B0009V1YR8 2869
B0043OYFKU 2477
B0000YUXI0 2143
B003V265QW 2088


#### 4. Câu hỏi và nhận xét:
- Trung bình mỗi product được đánh giá bao nhiêu lần? Product nào được đánh giá nhiều nhất và số lượng rating tối đa là bao nhiêu?
    - Trung bình mỗi product được đánh giá khoảng 8 lần, nhưng có product được đánh giá max 7,533 lần nên chênh lệch rất lớn.
- Bao nhiêu sản phẩm chỉ có 1 rating, và tỷ lệ so với tổng số product là bao nhiêu?
    - Có 103,484 product (~41.5%) chỉ được đánh giá 1 lần do đó nhiều sản phẩm ít được quan tâm.
- Top 5 product được đánh giá nhiều nhất là những sản phẩm nào, và số lượng rating của chúng so với phần còn lại như thế nào?
    - Top 5 sản phẩm được đánh giá nhiều nhất rất vượt trội so với phần còn lại, cho thấy bias về sản phẩm phổ biến trong dữ liệu.

## 7. Time activity

In [7]:
timestamps = data_raw[:,3].astype(int).astype('datetime64[s]')

years = timestamps.astype('datetime64[Y]').astype(int) + 1970
years_unique, years_counts = np.unique(years, return_counts=True)

print("Ratings per year:")
for y, c in zip(years_unique, years_counts):
    print(f"{y}: {c} ratings")

Ratings per year:
1998: 2 ratings
1999: 5 ratings
2000: 16 ratings
2001: 76 ratings
2002: 163 ratings
2003: 278 ratings
2004: 1308 ratings
2005: 3476 ratings
2006: 5990 ratings
2007: 17728 ratings
2008: 28862 ratings
2009: 41429 ratings
2010: 71236 ratings
2011: 147490 ratings
2012: 293339 ratings
2013: 798903 ratings
2014: 612769 ratings


#### Câu hỏi và nhận xét:
- Dữ liệu rating phân bố theo năm thế nào? Có năm nào có số lượng rating rất ít không?
    - Dataset có rating trải dài từ 1998 đến 2014, nhưng số lượng rating ban đầu rất ít (1998-2004 chỉ vài ngàn).
- Có sự tăng trưởng hay giảm sút rõ ràng theo năm không? Năm nào có peak số rating?
    - Từ 2010 trở đi số rating tăng mạnh, đạt đỉnh ~798,903 ratings năm 2013, sau đó giảm nhẹ năm 2014.
- Trend theo thời gian có thể phản ánh thay đổi hành vi người dùng hay sự phổ biến sản phẩm không?
    - Trend này cho thấy Amazon ngày càng được nhiều user đánh giá, với sự tăng trưởng nhanh trong những năm gần cuối.